# DETECTEUR DE PITCH TEMPS REEL

In [24]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft
from scipy.fftpack import rfft

from scipy import signal
import os
%matplotlib tk


In [25]:
dureeChunk = 0.2
RATE = 44100   #frequence d'échantillonage
CHUNK = int(dureeChunk *RATE) #longeur du tronçons de donnée à analyser, plus grand plus fft precise
FORMAT = pyaudio.paInt16
CHANNELS = 1
SEUIL = 20 #pour afficher le seuil,  en dB 
#PAS = 1024 * 4 # troncons que traite effectivemnt la fft avec nfft


alpha = 2**(1/12) #pas pour la gamme tempérée
note_grave = 220
Nb_Octave = 1
OCTAVE = np.zeros(13*Nb_Octave)
for i in range (13*Nb_Octave):
    OCTAVE[i]= note_grave*(alpha**i)

    
Freq_Min =note_grave #affichage de la freq min pour le temps-freq
Freq_Max = note_grave*(2**Nb_Octave)  #affichage de la freq max pour le temps-freq

freq_min_recherche = 220
freq_max_recherche = 440

TimeListen = 30 #largeur aproximatif d'écoute en s de l'analyse temps-freq

In [26]:
def calcul_periodogramme(x,Fe,duree_sous_bloc):
    ### périodogramme moyenné. Sections de 0.1 seconde
    N=len(x)
    N1=int(np.floor(duree_sous_bloc*Fe))
    K=int(np.floor(N/N1))
    w=signal.hamming(N1)
    ind1 = 0
    ind2 = N1-1
    b = w*x[ind1:(ind2+1)] 
    f, B = signal.periodogram(b, Fe,nfft =44100)
    B=B/N1
    while ind2+N1<N:
        ind1=ind1+N1
        ind2=ind2+N1
        b=w*x[ind1:(ind2+1)]
        f, aux = signal.periodogram(b, Fe, nfft =44100)
        B=B+aux/N1
    B=B/K
    B=10*np.log10(B)
    return(f,B)
    

In [27]:
def calcul_periodogramme(x,Fe,duree_sous_bloc):
    ### périodogramme moyenné. Sections de 0.1 seconde
    N=len(x)
    #N1=int(np.floor(duree_sous_bloc*Fe))
    
    w=signal.hamming(N)
    ind1 = 0
    ind2 = N-1
    b = w*x
    f, B = signal.periodogram(b, Fe,nfft =44100)
    B=B/N
    B[0]=0.1
    B=10*np.log10(B)
    return(f,B)

In [28]:
def Energie(x):
    E = 0
    for k in range(len(x)-1):
        E+=x[k]**2
    return 10*np.log10(E/len(x))

In [29]:

p = pyaudio.PyAudio()

#data du micro
stream = p.open(
    format = FORMAT ,
    channels = CHANNELS,
    rate = RATE,
    input= True,
    output = True,
    frames_per_buffer=CHUNK)



fig, (ax,axE,ax_fft,ax2) = plt.subplots(4,figsize=(10,7))



ax.set_xlim(0,CHUNK)
axE.set_xlim(0,CHUNK)
ax2.set_xlim(0,TimeListen*7.5)
ax_fft.set_xlim(0,RATE/20)



#variable pour plotter
x = np.arange(0, 1 * CHUNK ,1)
xE = np.arange(0, 1 * CHUNK ,1)
x2 = np.arange(0,1*CHUNK,1)
x_fft, aux = calcul_periodogramme(np.zeros(CHUNK),RATE,0.1)



line, = ax.plot(x,np.random.rand(CHUNK),'-',lw=2)
lineE, = axE.plot(xE,np.random.rand(CHUNK),'-',lw=2)
line_x_fft, = ax_fft.plot((x_fft),np.random.rand(len(x_fft)),'-',lw=2)
line_x2, = ax2.plot(x2,np.random.rand(CHUNK),'-',lw=2)
#plt.setp(ax_fft,xticks=[55,110,220,440,880,1760,3560])


ax.set_title("AUDIO FORME D'ONDE")

ax.set_ylabel('volume')
ax.set_ylim(-128,128)
#ax.set_xlim(0,CHUNK)



axE.set_title("Energie")

axE.set_ylabel('dB')
axE.set_ylim(0,40)
#ax.set_xlim(0,CHUNK)

ax_fft.set_title("AUDIO SPECTRE")
ax_fft.set_ylabel('Amplitude')
ax_fft.set_ylim(-100,25)
#ax_fft.set_xlim(0.001,CHUNK)


ax2.set_title("ANALYSE FREQ/TEMPS")
ax2.set_ylabel('freq')
plt.yticks(OCTAVE, ["La", "Sib", "Si","Do","Do#","Ré","Mib","Mi","Fa","Fa#","Sol","Sol#","La" ]*Nb_Octave)


plt.grid()
ax2.set_ylim(Freq_Min,Freq_Max)


m = np.zeros(CHUNK)
e =  np.zeros(CHUNK)
i = 0
max_initial = 0.00000001

while True:
    data = stream.read(CHUNK)#data en binaire
    data_int = np.array(struct.unpack(str(2*CHUNK)+ 'B', data),dtype='b')[::2] 
    line.set_ydata(data_int)
    
    
    print(Energie(data_int))
    lineE.set_ydata(Energie(data_int)*(e+1))
    if(Energie(data_int)>SEUIL):
        freq , periodo = calcul_periodogramme(data_int,RATE,0.5)
        line_x_fft.set_ydata(periodo) 
        cond = (freq >freq_min_recherche) * (freq <freq_max_recherche)
        periodo2 = (periodo+100)*cond
        ma = np.argmax(periodo2)
        pitch = freq[ma]
        m[i]= ma
        #print(m[i])
        line_x2.set_ydata(m)
        ax2.set_xlim(left = max (0, i-20), right = i+20)
    i= i+1

    
    fig.canvas.draw()
    fig.canvas.flush_events()



C:\Users\pierr\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log10
  del sys.path[0]


24.769045458172126
23.34770046847954
20.417595899874996
28.500808256781728
26.979623760873864
27.747429959109297
30.507897914694077
22.574695957681257
21.82928098492091
26.069694060583064
21.017907019017905
29.387075670472335
27.153119221463044
21.34434437292956
27.676764010048874
17.581907124217178
21.613041556759622
26.965540952649782
33.77657809343821
36.65477649720242
38.09963369174529
38.28811475869998
38.65832138838392
31.717171692937825
29.122184403799913


TclError: invalid command name "pyimage44"